In [1]:
import numpy 
with open('full_brain_volume_annotated.npy', 'rb') as file:
    full_brain_volume_annotated = numpy.load(file)

In [2]:
from cloudvolume import CloudVolume

cloudpath = 'file:///home/sherry/Projects/pipeline_utility/Litao/test_mesh'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint32', # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = full_brain_volume_annotated.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False, progress=False)
vol.commit_info()
vol[:, :, :] = full_brain_volume_annotated[:, :, :]

In [ ]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=4)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False, progress=True)
tq.insert(tasks)
tasks = tc.create_mesh_manifest_tasks(cloudpath)
tq.insert(tasks)
tq.execute(progress=True)

Downloading: 100%|██████████| 19/19 [00:02<00:00,  9.31it/s]

Downloading: 100%|██████████| 38/38 [00:07<00:00,  5.12it/s]

Downloading: 100%|██████████| 38/38 [00:12<00:00,  3.02it/s]

Downloading: 100%|██████████| 76/76 [00:10<00:00,  6.92it/s]

Mesh:   0%|          | 0/8 [00:00<?, ?it/s]